<a href="https://colab.research.google.com/github/Adrita2211/ML_Project/blob/object_detection/span_categorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy pandas scikit-learn matplotlib seaborn typer tqdm

In [2]:
import spacy
from spacy.lang.en.examples import sentences

nlp = spacy.load("en_core_web_sm")

###preprocessing data######

In [6]:
import pandas as pd
import re
import json
from pathlib import Path

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)  # Remove markdown links
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove special chars
    text = re.sub(r'\s+', ' ', text).strip()  # Collapse spaces
    return text

def preprocess_data(input_file, output_file):
    df = pd.read_csv(input_file)
    df['full_text'] = df['title'] + ". " + df['selftext']
    df['cleaned_text'] = df['full_text'].apply(clean_text)
    df = df[df['cleaned_text'].str.len() > 50]

    # Save to JSONL
    with open(output_file, 'w') as f:
        for text in df['cleaned_text']:
            json.dump({"text": text}, f)
            f.write('\n')

preprocess_data('/content/sample_data/reddit_climate_posts_2024.csv', '/content/sample_data/climate_posts.jsonl')

In [6]:
# !prodigy span.manual climate_spans climate_posts.jsonl --label CLIMATE_SOLUTION,POLICY,MITIGATION --patterns patterns.jsonl

/bin/bash: line 1: prodigy: command not found


In [4]:
!pip install spacy-annotator -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.8 MB/s eta 0:00:00


In [9]:
import spacy_annotator as spa

annotator = spa.Annotator(
    labels=["CLIMATE_SOLUTION", "CLIMATE_IMPACT", "POLICY", "SCIENCE", "IMPACTS","SCIENCE TERMS","MITIGATION"],
    model=nlp
)
#df = pd.read_json("/content/sample_data/climate_posts.jsonl", lines=True)
df=pd.read_csv("/content/sample_data/reddit_climate_posts_2024.csv")
df['full_text'] = df['title'] + ". " + df['selftext']
#df_labels = annotator.annotate(df=df, col_text="text", shuffle=True)
#df_labels
df['full_text']
df_labels = annotator.annotate(df=df, col_text="full_text", shuffle=True)
#df_labels

HTML(value='-1 examples annotated, 36 examples left')

Text(value='', description='CLIMATE_SOLUTION', layout=Layout(width='auto'), placeholder='ent one, ent two, ent…

Text(value='', description='CLIMATE_IMPACT', layout=Layout(width='auto'), placeholder='ent one, ent two, ent t…

Text(value='', description='POLICY', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='SCIENCE', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='IMPACTS', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='SCIENCE TERMS', layout=Layout(width='auto'), placeholder='ent one, ent two, ent th…

Text(value='', description='MITIGATION', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three…

Output()

In [10]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

# Load a blank English model (or 'en_core_web_sm' for pre-trained vectors)
nlp = spacy.blank("en")
doc_bin = DocBin()

# Process each row in the annotated DataFrame
for _, row in tqdm(df_labels.iterrows(), total=len(df_labels)):
    doc = nlp.make_doc(row["full_text"])
    spans = []

    # Add spans if they exist
    if "spans" in row and isinstance(row["spans"], list):
        for span in row["spans"]:
            # Create span object (adjust to your DataFrame structure)
            span_obj = doc.char_span(
                span["start"],
                span["end"],
                label=span["label"],
                alignment_mode="contract"  # Handles token boundaries
            )
            if span_obj is not None:  # Skip invalid spans
                spans.append(span_obj)

    doc.spans["sc"] = spans  # "sc" = Span Categorizer
    doc_bin.add(doc)

# Save to disk
doc_bin.to_disk("/content/sample_data/train.spacy")
print(f"Saved {len(doc_bin)} documents to /content/sample_data/train.spacy")

100%|██████████| 35/35 [00:00<00:00, 102.94it/s]

Saved 35 documents to /content/sample_data/train.spacy


In [11]:
!python -m spacy init config --lang en --pipeline spancat --optimize accuracy /content/config.cfg

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: spancat
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
!python -m spacy download en_core_web_lg -q
import spacy

# Verify vectors are loaded
nlp = spacy.load("en_core_web_lg")
print("Vectors loaded:", len(nlp.vocab.vectors))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Vectors loaded: 342918


In [15]:
!python -m spacy init config --lang en --pipeline spancat --optimize accuracy /content/config.cfg --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: spancat
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [16]:
!python -m spacy train /content/config.cfg --output /content/models --paths.train /content/sample_data/train.spacy --paths.dev /content/sample_data/train.spacy --gpu-id 0


ℹ Saving to output directory: /content/models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/usr/local/lib/python3.11/dist-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/typer/core.py", line 740, in main
    return _main(
           ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/typer/core.py", line 195, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1697, in 